In [2]:
#load spectres of templates at last point
import os
from astropy.table import Table
import numpy as np
import eazy
from datetime import datetime
import matplotlib.pyplot as plt
from numba import jit
from astropy.io import fits
import helper_module as hmod
from astropy.table import Table, join, hstack
import eazy_routines as ez

/home/t/miniconda3/envs/eazy39/lib/python3.9/site-packages/eazy/data/templates -> ./templates
/home/t/miniconda3/envs/eazy39/lib/python3.9/site-packages/eazy/data/filters/FILTER.RES.latest -> ./FILTER.RES.latest


In [3]:
############# SETTINGS #############
templCnt = 13
cCap = 0.1
doMultithread = False
threadCnt = 4
####################################
if not doMultithread: threadCnt = 1

In [4]:

#path = "templates-custom/45k/raw"
path = "templates-custom/EMlines"#!temp
#out_path = "templates-custom/45k"
out_path = "templates-custom"#!temp
path = os.path.dirname(os.path.realpath(__file__)) + "/" + path
out_path = os.path.dirname(os.path.realpath(__file__)) + "/" + out_path

templates_in = [f for f in os.listdir(path) if "bin1.fits" in f or "bin0.fits" in f or ".spec" in f]
spectras = []
for temp in templates_in:
    if ".fits" in temp:
        tab = Table.read(path+"/"+temp)
        flux = tab["flux"].T[-1]
    elif ".spec" in temp:
        tab = np.loadtxt(path+"/"+temp)
        flux = tab.T[-1]
    spectras.append(flux)
spectras = np.array(spectras)
for i in range(len(spectras)):
    #normalize
    spectras[i] = spectras[i] / np.sum(spectras[i])

NameError: name '__file__' is not defined

In [ ]:
#setup intermediate folderstructure and such
intermediateFolder = "spectras_temp"
if not os.path.exists(intermediateFolder):
    os.makedirs(intermediateFolder)
    print("Created intermediate folder: "+intermediateFolder)
for i in range(templCnt):
    if not os.path.exists(f'{intermediateFolder}/{i}.spec'):
        np.savetxt(f'{intermediateFolder}/{i}.spec', spectras[i])
        print("Created intermediate file: "+f'{intermediateFolder}/{i}.spec')

In [ ]:
#matrix stuff
matrix = np.random.uniform(0, 1, (templCnt, spectras.shape[0]))

@jit(nopython=True)
def matrixOp(matrix, spectras):
    vec = np.zeros((templCnt, 1))
    for i in range(templCnt):
        vec[i] = np.sum(matrix[i] * spectras[i])
    return vec


In [ ]:
############# FILTERS #############
flt = eazy.filters.FilterFile()

filts_nircam = {
        'F090W': 363,
        'F115W': 364,
        'F150W': 365,
        'F182M': 370,
        'F200W': 366,
        'F210M': 371,
        'F277W': 375,
        'F335M': 381,
        'F356W': 376,
        'F410M': 383,
        'F430M': 384,
        'F444W': 358,
        'F460M': 385,
        'F480M': 386
}

filts_HST = {
        'F105W': 202,
        'F125W': 203,
        'F140W': 204,
        'F160W': 205,
        'F435W': 233,
        'F606W': 214,
        'F775W': 216,
        'F814W': 239,
        'F850LP': 240
}

filts = {**filts_nircam, **filts_HST}

mw_reddening = ez.get_atten_dict(filts)

# get zeropoints
zps = [1.0]*len(filts)

In [ ]:
############# data loading #############
#load data
inname = "hlsp_jades_jwst_nircam_goods-s-deep_photometry_v1.0_catalog_large_withSpec.fits"
inpath = os.path.join(os.getenv('astrodata'), 'gds/jades/phot', inname)

# print the meta info
with fits.open(inpath) as hdul:
    print(hdul.info())

# load photometry table
tab = Table.read(inpath, hdu=6)
tab_redshifts = Table.read(inpath, hdu=9)

# load fluxes
# CIRC1: 0.10 arcsec aperture (see README)
ext = '_CIRC1'
cols_fluxes = hmod.get_matches(ext, tab.columns, exclude='_ei')
cols_f = np.sort(hmod.get_matches(ext, cols_fluxes, exclude='_e'))
cols_fe = np.sort(hmod.get_matches('_e', cols_fluxes))
cols_fluxes = list(np.vstack([cols_f, cols_fe]).T.flatten())
cols = list(np.insert(cols_fluxes, 0, ['ID', 'RA', 'DEC', 'z_spec']))

tab = join(tab, tab_redshifts['ID', 'z_spec'], join_type='inner', keys='ID')
tab_out = tab[cols]

# convert from nJy to uJy
# and apply MW reddening
keys = np.array(list(mw_reddening.keys()))
for c in cols_fluxes:
    tab_out[c].unit = u.nJy
    tab_out[c] = tab_out[c].to(u.uJy)
    
    # apply MW reddening
    matches = hmod.get_matches(keys, c, get_idxs=True)
    key = keys[np.int32(matches[:,0])][0]
    tab_out[c] *= mw_reddening[key]

# rename columns
for c in cols_f:
    cnew = c.replace(ext, '_flux')
    tab_out.rename_column(c, cnew)

for c in cols_fe:
    cnew = c.replace(ext+'_e', '_err')
    tab_out.rename_column(c, cnew)

# save EAZY table
tab_out.write('./data/gds_jades_eazy.fits', format='fits', overwrite=True)


In [ ]:
def meassureBadnessOfZ(matrix,spectras):
    raise NotImplementedError("EAZY not implemented yet")

In [ ]:
############# MAIN #############
#setup updating plot
plt.ion()
figure, axs = plt.subplots(2, 1, figsize=(10,10))
axs[1].set_yscale("log")
axs[1].set_xscale("log")
dummy1 = np.zeros((13, spectras.shape[0]))-3
dummy1[0][0] = 0.1
axisImg1 = axs[0].imshow(dummy1, cmap="hot")
lines = [axs[i].plot([], [])[0] for i in range(1,2)]
cbar1 = figure.colorbar(axisImg1, ax=axs[0])#, values=np.linspace(0,1,100))
figure.canvas.draw()

#put in name for computed matrix
matrixName = input("Optimized matrix name: ")

losses = []

#main loop
for i in range(1000):
    Ns = ["""random matrix with length of threadCnt"""]
    if not doMultithread:
        def testC(c,N,matrix,spectras):
            matrix = matrix + c*N
            return meassureBadnessOfZ(matrix,spectras)
        N = Ns[0]
        losses.append(testC(c,N,matrix,spectras))
        while True:#loop of finding c's
            c = {something}
            loss = testC(c,N,matrix,spectras)
            if loss < losses[-1]:
                losses.append(loss)
                break
    else:
        raise NotImplementedError("Multithreading not implemented yet")


In [ ]:
"""
        #initialize model
        #model = tf.Variable(np.random.uniform(0,1, size=(13, spectras.shape[0])), dtype=tf.float32)
        model = tf.Variable(np.zeros((13, spectras.shape[0]),dtype=float), dtype=tf.float32)
        for i in tf.range(model.shape[1]):
            model[i%13,i].assign(1)
        losses = []
        model = fixModel(model)
        for i in range(10000):
            loss = train_step(model, spectras)
            model = fixModel(model)
            if i % 10 == 0:
                print(loss.numpy())
            if i % 1 == 0 and i > 0:
                #do a matplotlib heatmap of matrix
                axisImg1.set_data(np.log(model.numpy()+lSque))
                dummy = performMatrixOperation(model, spectras)
                axisImg2.set_data(np.log(gridOut(dummy).numpy()+lSque))
                axisImg3.set_data(np.reshape(np.log(vecOut(dummy).numpy()), (1,13)))
                #axisImg2.set_data(gridOut(dummy).numpy()+lSque)
                #axisImg3.set_data(np.reshape(vecOut(dummy).numpy(), (1,13)))
                lines[0].set_data(np.arange(len(orthogonalities)), orthogonalities)
                lines[1].set_data(np.arange(len(rowStds)), rowStds)
                lines[2].set_data(np.arange(len(colStds)), colStds)
                axs[3].set_xlim(0, len(orthogonalities))
                axs[4].set_xlim(0, len(rowStds))
                axs[5].set_xlim(0, len(colStds))
                axs[3].set_ylim(np.min(orthogonalities), np.max(orthogonalities))
                axs[4].set_ylim(np.min(rowStds), np.max(rowStds))
                axs[5].set_ylim(np.min(colStds), np.max(colStds))
                figure.canvas.draw()
                figure.canvas.flush_events() 
            if i % 30 == 0:
                np.save(f"templates-custom/45k/optimized_matrix_{matrixName}.npy", model.numpy())
            print(".", end="")
            losses.append(loss.numpy())
            orthogonalities.append(orthogonalityMeassure(performMatrixOperation(model, spectras)))
            rowStds.append(rowwiseStandardDeviationMeassure(model))
            colStds.append(columnwiseStandardDeviationMeassure(model))
            if len(losses) > 1000:
                if abs(loss-losses[-100]) < 0.000005:
                    break
        #save model
        np.save(f"{path}/optimized_matrix_{matrixName}_score:{loss:.3f}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.npy", model.numpy())
        losseses.append(losses)
    except KeyboardInterrupt:
        print("KeyboardInterrupt")
        np.save(f"{path}/optimized_matrix_{matrixName}_score:{loss:.3f}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.npy", model.numpy())
        losseses.append(losses)
        

#show losses
import matplotlib.pyplot as plt
[plt.plot(losses) for losses in losseses]
plt.show()"""